<a href="https://colab.research.google.com/github/danielAdama/QABert/blob/main/SQUAD_QA_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip3 install transformers numpy --pre torch[dynamo] --force-reinstall --extra-index-url https://download.pytorch.org/whl/nightly/cu117

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/nightly/cu117
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 83.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 79.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 906.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 KB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.0/769.0 KB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 93.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 KB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.8/661.8 KB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2

In [1]:
import os
import requests
import json
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader
from tqdm import tqdm

In [2]:
# !pip3 install ipywidgets --user

In [3]:
URL = 'https://rajpurkar.github.io/SQuAD-explorer/dataset'

if (not os.path.exists('squad')):
    os.mkdir('squad')
    
    for file in ['train-v2.0.json', 'dev-v2.0.json']:
        response = requests.get(os.path.join(URL,file))
        with open(os.path.join('squad',file), 'wb') as doc:
            for chunk in response.iter_content(chunk_size=4):
                doc.write(chunk)

In [4]:
with open(os.path.join('squad','train-v2.0.json'), 'rb') as doc:
    squad_dict = json.load(doc)

In [5]:
squad_dict.keys()

dict_keys(['version', 'data'])

In [6]:
squad_dict['data'][:3]

[{'title': 'Beyoncé',
  'paragraphs': [{'qas': [{'question': 'When did Beyonce start becoming popular?',
      'id': '56be85543aeaaa14008c9063',
      'answers': [{'text': 'in the late 1990s', 'answer_start': 269}],
      'is_impossible': False},
     {'question': 'What areas did Beyonce compete in when she was growing up?',
      'id': '56be85543aeaaa14008c9065',
      'answers': [{'text': 'singing and dancing', 'answer_start': 207}],
      'is_impossible': False},
     {'question': "When did Beyonce leave Destiny's Child and become a solo singer?",
      'id': '56be85543aeaaa14008c9066',
      'answers': [{'text': '2003', 'answer_start': 526}],
      'is_impossible': False},
     {'question': 'In what city and state did Beyonce  grow up? ',
      'id': '56bf6b0f3aeaaa14008c9601',
      'answers': [{'text': 'Houston, Texas', 'answer_start': 166}],
      'is_impossible': False},
     {'question': 'In which decade did Beyonce become famous?',
      'id': '56bf6b0f3aeaaa14008c9602',
    

In [7]:
def read_data(file_name):
    
    with open(os.path.join('squad',file_name), 'rb') as doc:
        squad_dict = json.load(doc)
    
    contexts = []
    questions = []
    answers = []

    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                if 'plausible_answers' in qa.keys():
                    ans = 'plausible_answers'
                else:
                    ans = 'answers'
                for answer in qa[ans]:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)
                    
    return contexts, questions, answers

In [8]:
train_contexts, train_questions, train_answers = read_data('train-v2.0.json')
val_contexts, val_questions, val_answers = read_data('dev-v2.0.json')

In [9]:
train_contexts[:2]

['Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".',
 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead s

In [10]:
val_contexts[:2]

['The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.',
 'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates 

In [11]:
train_answers[:2]

[{'text': 'in the late 1990s', 'answer_start': 269},
 {'text': 'singing and dancing', 'answer_start': 207}]

In [12]:
def add_end_idx(answers, contexts):
    
    """Function to create the end indexs
    
    """
    
    for answer, context in zip(answers, contexts):
        gold_text = answer['text']
        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)
        if context[start_idx:end_idx] == gold_text:
            answer['answer_end'] = end_idx
        else:
            for n in [1,2]:
                if context[start_idx - n:end_idx - n] == gold_text:
                    answer['answer_start'] = start_idx - n
                    answer['answer_end'] = end_idx - n

In [13]:
add_end_idx(train_answers, train_contexts)
add_end_idx(val_answers, val_contexts)

In [14]:
train_answers[:2]

[{'text': 'in the late 1990s', 'answer_start': 269, 'answer_end': 286},
 {'text': 'singing and dancing', 'answer_start': 207, 'answer_end': 226}]

# Encoding Text

In [15]:
CHECKPOINT = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT)

**The following assertion ensures that our tokenizer is a fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, and we will need some of the special features they have for our preprocessing.**

In [16]:
tokenizer.is_fast

True

In [17]:
tokenizer("What is your name?", "My name is Daniel Adama.")

{'input_ids': [101, 2054, 2003, 2115, 2171, 1029, 102, 2026, 2171, 2003, 3817, 4205, 2050, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

To see how this works using the current example, we can limit the length to **100** and use a sliding window of **50 tokens**.

In [18]:
train_questions[9]

"What role did Beyoncé have in Destiny's Child?"

In [19]:
train_contexts[9]

'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".'

In [20]:
train_answers[9]

{'text': 'lead singer', 'answer_start': 290, 'answer_end': 301}

* max_length to set the maximum length (here 100)
* truncation="only_second" to truncate the context (which is in the second position) when the question with its context is too long
* stride to set the number of overlapping tokens between two successive chunks (here 50)
* return_overflowing_tokens=True to let the tokenizer know we want the overflowing tokens

In [21]:
inputs = tokenizer(
    train_questions[9],
    train_contexts[9],
    max_length=100,
    truncation='only_second',
    stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=True
)

for ids in inputs['input_ids']:
    print(tokenizer.decode(ids))

[CLS] what role did beyonce have in destiny's child? [SEP] beyonce giselle knowles - carter ( / biːˈjɒnseɪ / bee - yon - say ) ( born september 4, 1981 ) is an american singer, songwriter, record producer and actress. born and raised in houston, texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of r & b girl - group destiny's child. managed [SEP]
[CLS] what role did beyonce have in destiny's child? [SEP], record producer and actress. born and raised in houston, texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of r & b girl - group destiny's child. managed by her father, mathew knowles, the group became one of the world's best - selling girl groups of all time. their hiatus saw the release of beyonce's debut album [SEP]
[CLS] what role did beyonce have in destiny's child? [SEP] singer of r & b girl - group destiny's child. managed by 

As we can see, our example has been in split into four inputs, each of them containing the question and some part of the context. Note that the answer to the question (“**lead singer**”) only appears in the first and second inputs, so by dealing with long contexts in this way we will create some training examples where the answer is not included in the context.

In [22]:
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

Because here we only tokenized one example, we get a list of 0s:

In [23]:
inputs["overflow_to_sample_mapping"]

[0, 0, 0, 0]

In [24]:
inputs = tokenizer(
    train_questions[2:6],
    train_contexts[2:6],
    max_length=100,
    truncation='only_second',
    stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=True
)


print(f"The 4 examples gave {len(inputs['input_ids'])} features.")
print(f"Here is where each comes from: {inputs['overflow_to_sample_mapping']}.")

The 4 examples gave 15 features.
Here is where each comes from: [0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3].


In [25]:
ans = train_answers[2:6]
start_positions = []
end_positions = []


for i, offset in enumerate(inputs['offset_mapping']):
    sample_idx = inputs["overflow_to_sample_mapping"][i]
    answer = ans[sample_idx]
    start_char = answer['answer_start']
    end_char = answer['answer_start'] + len(answer['text'])
    sequence_ids = inputs.sequence_ids(i)
    
    # Find the start and end of the context
    idx = 0
    while sequence_ids[idx] != 1:
        idx += 1
    context_start = idx
    
    while sequence_ids[idx] == 1:
        idx += 1
    context_end = idx - 1
    
    # If the answer is not fully inside the context, label is (0, 0)
    if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
        start_positions.append(0)
        end_positions.append(0)
    else:
        # Otherwise it's the start and end token positions
        idx = context_start
        while idx <= context_end and offset[idx][0] <= start_char:
            idx += 1
        start_positions.append(idx - 1)
        
        idx = context_end
        while idx >= context_start and offset[idx][1] >= end_char:
            idx -= 1
        end_positions.append(idx + 1)


start_positions, end_positions

([0, 0, 77, 44, 58, 21, 0, 0, 78, 39, 0, 94, 59, 24, 0],
 [0, 0, 77, 44, 60, 23, 0, 0, 79, 40, 0, 97, 62, 27, 0])

Let’s take a look at a few results to verify that our approach is correct. For the fifth feature we find (58, 60) as labels, so let’s compare the theoretical answer with the decoded span of tokens from 58 to 60 (inclusive):

In [26]:
idx = 4
sample_idx = inputs["overflow_to_sample_mapping"][idx]
answer = ans[sample_idx]["text"]

start = start_positions[idx]
end = end_positions[idx]
labeled_answer = tokenizer.decode(inputs["input_ids"][idx][start : end + 1])

print(f"Theoretical answer: {answer} \nLabels give: {labeled_answer}")

Theoretical answer: Houston, Texas 
Labels give: houston, texas


So that’s a match! Now let’s check index 1, where we set the labels to (0, 0), which means the answer is not in the context chunk of that feature:

In [27]:
idx = 1
sample_idx = inputs["overflow_to_sample_mapping"][idx]
answer = ans[sample_idx]["text"]

decoded_example = tokenizer.decode(inputs["input_ids"][idx])
print(f"Theoretical answer: {answer}, \nDecoded example: {decoded_example}")

Theoretical answer: 2003, 
Decoded example: [CLS] when did beyonce leave destiny's child and become a solo singer? [SEP] singer, songwriter, record producer and actress. born and raised in houston, texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of r & b girl - group destiny's child. managed by her father, mathew knowles, the group became one of the world's best - selling girl groups of all time. their hiatus saw the release [SEP]


For this notebook to work with any kind of models, we need to account for the special case where the model expects padding on the left (in which case we switch the order of the question and the context)

In [28]:
questions = [q.strip() for q in train_questions]
print(questions[:5])

['When did Beyonce start becoming popular?', 'What areas did Beyonce compete in when she was growing up?', "When did Beyonce leave Destiny's Child and become a solo singer?", 'In what city and state did Beyonce  grow up?', 'In which decade did Beyonce become famous?']


Note that we defined two constants to determine the maximum length used as well as the length of the sliding window, and that we added a tiny bit of cleanup before tokenizing: some of the questions in the SQuAD dataset have extra spaces at the beginning and the end that don’t add anything (and take up space when being tokenized if you use a model like RoBERTa), so we removed those extra spaces.

In [29]:
max_length = 384 # The maximum length of a feature (question and context)
stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

def preprocess_pipeline(questions, contexts, answers):
    
    """Function to prepare the data in the right format to feed to the model
    """
    
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    questions = [q.strip() for q in questions]
    
    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    inputs = tokenizer(
        questions,
        contexts,
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = inputs.pop("offset_mapping")
    
    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_map = inputs.pop("overflow_to_sample_mapping")
    ans = answers
    start_positions = []
    end_positions = []


    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = ans[sample_idx]
        start_char = answer['answer_start']
        end_char = answer['answer_start'] + len(answer['text'])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx

        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)
            
            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    
    return inputs

In [30]:
# The dataset is too large so i split to build my pipeline before proceeding
train_encodings = preprocess_pipeline(train_questions[:10000], train_contexts[:10000], train_answers[:10000])
val_encodings = preprocess_pipeline(val_questions[:10000], val_contexts[:10000], val_answers[:10000])

To apply this function to the whole training set, we use the .map() method with the batched=True flag. It’s necessary here as we are changing the length of the dataset (since one example can give several training features):

In [31]:
train_encodings.keys()

dict_keys(['input_ids', 'attention_mask', 'start_positions', 'end_positions'])

In [32]:
# train_encodings['start_positions']

In [33]:
class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [34]:
train_datasets = SquadDataset(train_encodings)
val_datasets = SquadDataset(val_encodings)

# Fine-tuning the model

In [35]:
model = AutoModelForQuestionAnswering.from_pretrained(CHECKPOINT)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this mode

In [36]:
model

DistilBertForQuestionAnswering(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
      

In [40]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
model.train()
opt = AdamW(model.parameters(), lr=2e-5)

In [41]:
train_loader = DataLoader(train_datasets, batch_size=8, shuffle=True)

In [42]:
## We are training the model on 2 Epochs
for epoch in range(2):
  loop = tqdm(train_loader)
  for batch in loop:
    opt.zero_grad()

    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    start_positions = batch['start_positions'].to(device)
    end_positions = batch['end_positions'].to(device)


    outputs = model(
        input_ids,
        attention_mask=attention_mask,
        start_positions=start_positions,
        end_positions=end_positions
    )

    ## Calculate loss for every parameter
    loss = outputs[0]
    ## Gradient update
    loss.backward()
    ## Update those gradients
    opt.step()

    loop.set_description(f'Epoch {epoch}')
    loop.set_postfix(loss=loss.item())

Epoch 1: 100%|██████████| 1268/1268 [06:14<00:00,  3.38it/s, loss=0.923]
